In [2]:
#raspa tela
import requests
from bs4 import BeautifulSoup
#dataframe
import pandas as pd
#gráficos
import matplotlib.pyplot as plt
#data e hora.
import datetime as dtm
from datetime import date as dt
import time
import re
import numpy as np

# Informando período

In [3]:
#definir quais anos você quer analisar
AnoInicial=int(input("Informe o Ano inicial: "))
AnoFinal=int(input("Informe o Ano final: "))
print("Lembrando que quanto mais anos, mais demorado será a extração")

Informe o Ano inicial:  2023
Informe o Ano final:  2023


Lembrando que quanto mais anos, mais demorado será a extração


# Processo de Web Scraping

In [4]:
tempo_inicio = time.time()
linkRaiz = 'https://www.cbf.com.br/futebol-brasileiro/competicoes/campeonato-brasileiro-serie-a'
listaLinks = []
ListaFinal = []
jogo=[]
Info = []
Rodada = []
j = []
x = []

for ano in range(AnoInicial, AnoFinal+1):
    for nj in range(300,303):
        linkJogo = f"{linkRaiz}/{ano}/{nj}"
        listaLinks.append(linkJogo)
        valorweb = requests.get(linkJogo)
        soup = BeautifulSoup(valorweb.content, 'html.parser')

        
        title = soup.title.text
        jogos=soup.find(class_='color-white block text-1').text
        jogo.append(jogos)
        ListaFinal.append(title)
        localdthr_jogo = soup.find_all(class_='text-2 p-r-20')
        Texto = [element.get_text(strip=True) for element in localdthr_jogo]
        Info.append(Texto)
        jogos=soup.find(class_='color-white block text-1').text
        jogos = jogos.split(':')[1].strip()
        

        GolJog = soup.find_all(class_='hidden-sm hidden-md hidden-lg m-t-20')
        i = [element.get_text(strip=True) for element in GolJog]
        j.extend(i)
        
        captTimes = soup.find_all(class_='time-nome color-white')
        Texto = [element.get_text(strip=True) for element in captTimes]
        x.extend(Texto)
           

matriz = [[i_item, Texto_item] for i_item, Texto_item in zip(j,x) if j and x != '']

tempo_final = time.time()
tempo_decorrido = tempo_final - tempo_inicio


print(f'Tempo decorrido: {tempo_decorrido:.2f} segundos')

Tempo decorrido: 3.82 segundos


# Iniciar o Processo de DataFrame para acompanhamento dos Jogos

In [5]:
Local = []
Data = []
Hora = []
Cidade = []
Estadio = []
UF = []
Dia = []
for Texto in Info:
    local = Texto[0]
    Local.append(local)
    data = Texto[1]
    hora = Texto[2]
    Hora.append(hora)
    dia = data.split(',')
    x = dia[0]
    Dia.append(x)
    i = dia[1]
    Data.append(i)

for i in Local:
    x = i.split('-')
    y = x[0]
    z = x[1]
    w = x[2]
    Estadio.append(y)
    Cidade.append(z)
    UF.append(w)


In [6]:
#Coletar Número Jogo
NumeroJogo=[]
for i in jogo:
    j=i.split('Jogo:')[1].strip()
    NumeroJogo.append(j)


#Coletar Time da Casa
time_casa = []
UF_Time_Mandante = []
for partidas in ListaFinal:
    times = partidas.split(' x ')
    timeCasa = times[0].strip()  # Remover espaços extras no início e no final
    nome_time_casa = timeCasa.split(' - ')[0].strip()  # Separar o nome do time da parte do estado
    Estado= timeCasa.split(' - ')[1].strip()
    Estado = Estado.split(' ')[0]
    time_casa.append(nome_time_casa )
    UF_Time_Mandante.append(Estado)


#coletar Visitante
time_visitante = []
UF_Time_Visitante = []
for partidas in ListaFinal:
    times = partidas.split(' x ')
    timeVisitante = times[1].strip()  # Remover espaços extras no início e no final
    nome_time_visitante = timeVisitante.split(' - ')[0].strip() 
    nome_time_visitante1 = nome_time_visitante[2:]  # Remover o número no início
    time_visitante.append(nome_time_visitante1)
    Estado = timeVisitante.split('-')[1].strip()
    UF_Time_Visitante.append(Estado)



#coletar gols mandante
gols_mandante = []

for gols in ListaFinal:
    gol_mandante = gols.split(' x ')[0].strip()  # Extrair os gols do time da casa
    time = gol_mandante.split(' - ')[1].strip()
    time=time[3:]
    gols_mandante.append(time)

#Coletar gols visitante
gols_visitante=[]
for gols in ListaFinal:
    gols_visitantes=gols.split(' x ')[1].strip()
    gols_visitantes=gols_visitantes.split(' - ')[0].strip()
    gols_visitantes=gols_visitantes.split(' ')[0]
    gols_visitante.append(gols_visitantes)

#Coletar Ano
anoJogo=[]
for i in ListaFinal:
    Ano_Jogos=i.split(' x ')[1].strip()
    Ano_Jogos=Ano_Jogos.split(' - ')[3]
    Ano_Jogos=Ano_Jogos[-4:]
    anoJogo.append(Ano_Jogos)

In [7]:
#Determinar quem venceu
def vitoria(gols_mandante, gols_visitante):
    if gols_mandante > gols_visitante:
        return 'Mandante'
    elif gols_mandante < gols_visitante:
        return 'Visitante'
    else:
        return 'Empate'
resultado=[vitoria(gols_mandante,gols_visitante) for gols_mandante,gols_visitante in zip(gols_mandante,gols_visitante)]

In [8]:
#Criando o DataFrame 
data = {'NumeroJogo':NumeroJogo, 'UF':UF, 'Estadio':Estadio, 'Cidade':Cidade, 'Hora':Hora, 'Data':Data, 'Quem Venceu?':resultado, 'AnoJogo':anoJogo, 'Time Mandante':time_casa,'UF Mandante': UF_Time_Mandante , 'Gols Mandante':gols_mandante,'Time Visitante':time_visitante,'UF Visitante': UF_Time_Visitante,'Gols Visitante':gols_visitante}
df = pd.DataFrame(data)
df

,NumeroJogo,UF,Estadio,Cidade,Hora,Data,Quem Venceu?,AnoJogo,Time Mandante,UF Mandante,Gols Mandante,Time Visitante,UF Visitante,Gols Visitante
0,300,MG,Independência,Belo Horizonte,19:00,28 de Outubro de 2023,Visitante,2023,America Saf,MG,3,Grêmio,RS,4
1,301,DF,Mané Garrincha,Brasilia,20:00,01 de Novembro de 2023,Visitante,2023,Flamengo,RJ,1,Santos,SP,2
2,302,RJ,Nilton Santos,Rio de Janeiro,21:30,01 de Novembro de 2023,Visitante,2023,Botafogo,RJ,3,Palmeiras,SP,4


In [9]:
#Excluindo jogos que ainda não ocorreram 
df['Gols Visitante'] = pd.to_numeric(df['Gols Visitante'], errors='coerce').astype('Int64')
df['Gols Mandante'] = pd.to_numeric(df['Gols Mandante'], errors='coerce').astype('Int64')
df['NumeroJogo'] = pd.to_numeric(df['NumeroJogo'], errors='coerce').astype('Int64')
df['Rodada'] = df['NumeroJogo'].apply(lambda num_jogo: int(num_jogo / 10) if num_jogo % 10 == 0 else int(num_jogo // 10) + 1)
df['Time Mandante'] = df['Time Mandante'].str.replace('.', '')
df['Time Visitante'] = df['Time Visitante'].str.replace('.', '')
df = df.dropna(subset=['Gols Visitante'])
X = ['NumeroJogo', 'Estadio', 'Cidade', 'UF', 'Hora', 'Data', 'Quem Venceu?', 'AnoJogo', 'Rodada', 'Time Mandante', 'UF Mandante', 'Gols Mandante', 'Time Visitante', 'UF Visitante', 'Gols Visitante']
df = df[X]
df

,NumeroJogo,Estadio,Cidade,UF,Hora,Data,Quem Venceu?,AnoJogo,Rodada,Time Mandante,UF Mandante,Gols Mandante,Time Visitante,UF Visitante,Gols Visitante
0,300,Independência,Belo Horizonte,MG,19:00,28 de Outubro de 2023,Visitante,2023,30,America Saf,MG,3,Grêmio,RS,4
1,301,Mané Garrincha,Brasilia,DF,20:00,01 de Novembro de 2023,Visitante,2023,31,Flamengo,RJ,1,Santos,SP,2
2,302,Nilton Santos,Rio de Janeiro,RJ,21:30,01 de Novembro de 2023,Visitante,2023,31,Botafogo,RJ,3,Palmeiras,SP,4


In [10]:
#Exportando Para Excel
df.to_excel(f"CampeonatoBrasileiro_Jogos_CBF_{AnoInicial}-{AnoFinal}_SerieA.xlsx", index = False)

### Acompanhar desempenho dos times Individualmente

In [11]:
#Copiar Dataframe anterior
dx_local = df[['NumeroJogo', 'Time Mandante','UF Mandante', 'Gols Mandante', 'Quem Venceu?','Estadio', 'Cidade','UF', 'Hora', 'Data', 'AnoJogo']].copy()
dx_visitante = df[['NumeroJogo', 'Time Visitante','UF Visitante', 'Gols Visitante', 'Quem Venceu?', 'Estadio', 'Cidade','UF', 'Hora', 'Data','AnoJogo']].copy()
dx_local.columns = ['NumeroJogo', 'Time','UF Time', 'Gols', 'Quem Venceu?','Estadio', 'Cidade','UF', 'Hora', 'Data', 'AnoJogo']
dx_local['Mandante/Visitante'] = 'Mandante'
dx_visitante.columns = ['NumeroJogo', 'Time','UF Time', 'Gols', 'Quem Venceu?', 'Estadio', 'Cidade','UF', 'Hora', 'Data', 'AnoJogo']
dx_visitante['Mandante/Visitante'] = 'Visitante'
#concatenar
dx = pd.concat([dx_local, dx_visitante], ignore_index=True)
#Demonstrar pontuação
dx['Pontos'] = dx.apply(lambda row: 3 if row['Quem Venceu?'] == row['Mandante/Visitante'] else (1 if row['Quem Venceu?'] == 'Empate' else 0), axis=1)
dx = dx.drop('Quem Venceu?', axis=1)
dx

,NumeroJogo,Time,UF Time,Gols,Estadio,Cidade,UF,Hora,Data,AnoJogo,Mandante/Visitante,Pontos
0,300,America Saf,MG,3,Independência,Belo Horizonte,MG,19:00,28 de Outubro de 2023,2023,Mandante,0
1,301,Flamengo,RJ,1,Mané Garrincha,Brasilia,DF,20:00,01 de Novembro de 2023,2023,Mandante,0
2,302,Botafogo,RJ,3,Nilton Santos,Rio de Janeiro,RJ,21:30,01 de Novembro de 2023,2023,Mandante,0
3,300,Grêmio,RS,4,Independência,Belo Horizonte,MG,19:00,28 de Outubro de 2023,2023,Visitante,3
4,301,Santos,SP,2,Mané Garrincha,Brasilia,DF,20:00,01 de Novembro de 2023,2023,Visitante,3
5,302,Palmeiras,SP,4,Nilton Santos,Rio de Janeiro,RJ,21:30,01 de Novembro de 2023,2023,Visitante,3


### Informação de Gols por jogador e time

In [44]:
x = np.array(matriz)
y = np.array(NumeroJogo)
j = np.array(anoJogo)
matriz1_reshaped = x.reshape(-1, 2, 2)

resultado = []

# Use um índice para rastrear o número do jogo da matriz 2
indice_matriz2 = 0

# Itere sobre a matriz 1 agrupando os registros em jogos e adicionando o número do jogo
for i in range(0, len(x), 2):
    jogo = x[i:i + 2]
    ano = j[indice_matriz2]  # Use a variável "ano" em vez de "Anojogo"
    numero_jogo = y[indice_matriz2]
    resultado.append([numero_jogo, jogo, ano])  # Use "ano" em vez de "Anojogo"
    indice_matriz2 += 1


numeros_jogo = []
detalhes_jogo = []
anos = []

# Itere sobre o resultado
for item in resultado:
    numeros_jogo.append(item[0])
    detalhes_jogo.append(item[1])
    anos.append(item[2])


In [45]:
numeros_jogo = [item[0] for item in resultado]
detalhes_jogo = [item[1] for item in resultado]
anos = [item[2] for item in resultado]


In [46]:

lista_de_listas = []
# Itere pelos dados originais e crie uma nova lista para cada elemento
for elemento in detalhes_jogo:
    nova_lista = [item for sublist in elemento for item in sublist]
    lista_de_listas.append(nova_lista)
lista_de_listas = list(zip(numeros_jogo, lista_de_listas, anos))
print(lista_de_listas)

[('300', ["Rodrigo 3' (1ºT)Gonzalo Mastriani 34' (1ºT)Gonzalo Mastriani 3' (2ºT)", 'America Saf - MG', "Reinaldo 16' (1ºT)Galdino 19' (1ºT)Suarez 25' (2ºT)Cristaldo 32' (2ºT)", 'Grêmio - RS'], '2023'), ('301', ["Pedro 20' (1ºT)", 'Flamengo - RJ', "Nonato 32' (1ºT)Joaquim 44' (2ºT)", 'Santos - SP'], '2023'), ('302', ["Carlos 20' (1ºT)Tche Tche 30' (1ºT)Junior 35' (1ºT)", 'Botafogo - RJ', "Endrick 4' (2ºT)Endrick 38' (2ºT)Lopez 44' (2ºT)Murilo 45+8' (2ºT)", 'Palmeiras - SP'], '2023')]


In [51]:
detalhe = []
times = []
jogo = []
temporada = []


for elemento in lista_de_listas:
    numero_jogo = elemento[0]
    gols_detalhes = elemento[1]
    detalhe.extend(gols_detalhes)
    ano = elemento[2]
   

temp = []
nomes_jogadores = []
nomes_times = []      
for i, dado in enumerate(detalhe):
    
    if i % 2 == 0:
        nomes = dado.split(')')
        nomes = [nome.strip() + ')' for nome in nomes if nome.strip()]
        nomes_jogadores.extend(nomes)
        temporada.extend([tempo] * len(nomes))
    else:
        # Multiplicar o nome do time pelo número de jogadores
        nomes_times.extend([dado] * len(nomes))


temp

[]

In [50]:
temporada

["Rodrigo 3' (1ºT)Gonzalo Mastriani 34' (1ºT)Gonzalo Mastriani 3' (2ºT)",
 "Rodrigo 3' (1ºT)Gonzalo Mastriani 34' (1ºT)Gonzalo Mastriani 3' (2ºT)",
 "Rodrigo 3' (1ºT)Gonzalo Mastriani 34' (1ºT)Gonzalo Mastriani 3' (2ºT)",
 "Reinaldo 16' (1ºT)Galdino 19' (1ºT)Suarez 25' (2ºT)Cristaldo 32' (2ºT)",
 "Reinaldo 16' (1ºT)Galdino 19' (1ºT)Suarez 25' (2ºT)Cristaldo 32' (2ºT)",
 "Reinaldo 16' (1ºT)Galdino 19' (1ºT)Suarez 25' (2ºT)Cristaldo 32' (2ºT)",
 "Reinaldo 16' (1ºT)Galdino 19' (1ºT)Suarez 25' (2ºT)Cristaldo 32' (2ºT)",
 "Pedro 20' (1ºT)",
 "Nonato 32' (1ºT)Joaquim 44' (2ºT)",
 "Nonato 32' (1ºT)Joaquim 44' (2ºT)",
 "Carlos 20' (1ºT)Tche Tche 30' (1ºT)Junior 35' (1ºT)",
 "Carlos 20' (1ºT)Tche Tche 30' (1ºT)Junior 35' (1ºT)",
 "Carlos 20' (1ºT)Tche Tche 30' (1ºT)Junior 35' (1ºT)",
 "Endrick 4' (2ºT)Endrick 38' (2ºT)Lopez 44' (2ºT)Murilo 45+8' (2ºT)",
 "Endrick 4' (2ºT)Endrick 38' (2ºT)Lopez 44' (2ºT)Murilo 45+8' (2ºT)",
 "Endrick 4' (2ºT)Endrick 38' (2ºT)Lopez 44' (2ºT)Murilo 45+8' (2ºT)",
 

In [16]:

#padrao = r'([\w\s]+)\s(\d{1,2})\'\s\((\d{1,2}[+-]?\d*\w*)\)'  esse não estava considerando gols nos acréscimos 

#Criando Regex através de Ferramenta de identificação de padrões 
padrao = r'([\w\s]+)\s(\d{1,2}\+?\d*)\'\s\((\d{1,2}[+-]?\d*\w*)\)'
# Lista para armazenar os resultados
resultados = []
Atleta = []
Minutos = []
Tempo = []
Minutos_Acrescido = []
# Loop sobre as strings
for s in nomes_jogadores:
    correspondencias = re.search(padrao, s)
    if correspondencias:
        nome = correspondencias.group(1)
        minuto = correspondencias.group(2)
        tempo = correspondencias.group(3)
        resultados.append({'Nome': nome, 'Minuto': minuto, 'Tempo': tempo})
        Atleta.append(nome)
        Minutos.append(minuto)
        Tempo.append(tempo)


for i in Minutos:
    x = i.split('+')
    z = int(x[0])
    
    for acrescimo in x[1:]:
        z += int(acrescimo)

    Minutos_Acrescido.append(z)



df = pd.DataFrame({'Nome': Atleta, 'Minuto':Minutos_Acrescido,'Tempo':Tempo, 'Time': nomes_times})

#Exiba o DataFrame
print(df)


                 Nome  Minuto Tempo              Time
0             Rodrigo       3   1ºT  America Saf - MG
1   Gonzalo Mastriani      34   1ºT  America Saf - MG
2   Gonzalo Mastriani       3   2ºT  America Saf - MG
3            Reinaldo      16   1ºT       Grêmio - RS
4             Galdino      19   1ºT       Grêmio - RS
5              Suarez      25   2ºT       Grêmio - RS
6           Cristaldo      32   2ºT       Grêmio - RS
7               Pedro      20   1ºT     Flamengo - RJ
8              Nonato      32   1ºT       Santos - SP
9             Joaquim      44   2ºT       Santos - SP
10             Carlos      20   1ºT     Botafogo - RJ
11          Tche Tche      30   1ºT     Botafogo - RJ
12             Junior      35   1ºT     Botafogo - RJ
13            Endrick       4   2ºT    Palmeiras - SP
14            Endrick      38   2ºT    Palmeiras - SP
15              Lopez      44   2ºT    Palmeiras - SP
16             Murilo      53   2ºT    Palmeiras - SP
